# 🇮🇳 Mobility Signal Index (MSI) Analysis
## Detecting Redistribution Patterns in Aadhaar Data

---

### Problem Statement

**Objective:** Detect "redistribution-like patterns" in Aadhaar enrollment and update data without overclaiming actual population movement.

**Key Insight:** When one geographic area shows *declining* Aadhaar activity while neighboring areas show *increasing* activity in the same time window, this creates a statistical signal that may indicate population redistribution.

### Methodology Overview

1. **Mobility Signal Index (MSI):** Measures inverse correlation between an area and its geographic neighbors
2. **Wave Propagation Detection:** Identifies patterns that spread spatially over time (start → peak → fade)
3. **Local Effect Filtering:** Distinguishes true redistribution from operational effects (e.g., single-pincode camps)

### Careful Wording

- ✅ "Redistribution-like patterns detected"
- ✅ "Possible population movement signals"
- ❌ "People definitely moved from A to B"

---

**Theme:** 🇮🇳 Indian Tricolor (Saffron • White • Green)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📦 IMPORTS & CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Statistical analysis
from scipy import stats
from collections import defaultdict
from datetime import datetime

# Display settings
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)
pio.renderers.default = 'notebook'

print("✅ Libraries loaded successfully!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🎨 INDIAN TRICOLOR THEME CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════════════

INDIA_COLORS = {
    # Primary tricolor
    'saffron': '#FF9933',           # केसरी - Courage & Sacrifice
    'saffron_light': '#FFB366',
    'saffron_dark': '#E67300',
    'white': '#FFFFFF',              # शांति - Peace & Truth
    'white_off': '#F8F9FA',
    'green': '#138808',              # हरा - Fertility & Prosperity  
    'green_light': '#1DB954',
    'green_dark': '#0D5C06',
    
    # Ashoka Chakra
    'navy': '#000080',
    'chakra_blue': '#0000CD',
    
    # Background & UI
    'background': '#0A0A1A',
    'background_light': '#1A1A2E',
    'text': '#F0F0F0',
    'text_muted': '#A0A0A0',
    'grid': '#2A2A4A',
    'accent': '#FFD700'  # Gold accent
}

# Indian tricolor gradient for continuous data
INDIA_COLORSCALE = [
    [0.0, INDIA_COLORS['green_dark']],
    [0.25, INDIA_COLORS['green']],
    [0.5, INDIA_COLORS['white']],
    [0.75, INDIA_COLORS['saffron']],
    [1.0, INDIA_COLORS['saffron_dark']]
]

# Custom Plotly template
INDIA_TEMPLATE = go.layout.Template(
    layout=go.Layout(
        paper_bgcolor=INDIA_COLORS['background'],
        plot_bgcolor=INDIA_COLORS['background'],
        font=dict(
            family="'Segoe UI', 'Helvetica Neue', Arial, sans-serif",
            color=INDIA_COLORS['text'],
            size=12
        ),
        title=dict(
            font=dict(size=24, color=INDIA_COLORS['saffron']),
            x=0.5,
            xanchor='center'
        ),
        xaxis=dict(
            gridcolor=INDIA_COLORS['grid'],
            linecolor=INDIA_COLORS['grid'],
            zerolinecolor=INDIA_COLORS['grid'],
            tickfont=dict(color=INDIA_COLORS['text_muted'])
        ),
        yaxis=dict(
            gridcolor=INDIA_COLORS['grid'],
            linecolor=INDIA_COLORS['grid'],
            zerolinecolor=INDIA_COLORS['grid'],
            tickfont=dict(color=INDIA_COLORS['text_muted'])
        ),
        colorway=[
            INDIA_COLORS['saffron'],
            INDIA_COLORS['green'],
            INDIA_COLORS['chakra_blue'],
            INDIA_COLORS['saffron_light'],
            INDIA_COLORS['green_light'],
            INDIA_COLORS['accent']
        ],
        hoverlabel=dict(
            bgcolor=INDIA_COLORS['background_light'],
            font_size=12,
            font_family="'Segoe UI', Arial"
        )
    )
)

print("🎨 Indian Tricolor Theme configured!")
print(f"   🟠 Saffron: {INDIA_COLORS['saffron']}")
print(f"   ⚪ White:   {INDIA_COLORS['white']}")
print(f"   🟢 Green:   {INDIA_COLORS['green']}")

---

## 1️⃣ Data Loading & Preprocessing

Loading three Aadhaar datasets:
- **Enrolment:** New Aadhaar registrations by age group
- **Demographic Updates:** Changes to name, address, DOB, etc.
- **Biometric Updates:** Fingerprint, iris, face updates

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📂 DATA LOADING
# ═══════════════════════════════════════════════════════════════════════════════

BASE_PATH = Path(".")

print("🔄 Loading Aadhaar datasets...")
print("=" * 60)

# Load Enrolment Data
enrolment_files = sorted(BASE_PATH.glob("api_data_aadhar_enrolment/*.csv"))
enrolment_df = pd.concat([pd.read_csv(f) for f in enrolment_files], ignore_index=True)
print(f"📋 Enrolment:   {len(enrolment_df):>12,} records")

# Load Demographic Data  
demographic_files = sorted(BASE_PATH.glob("api_data_aadhar_demographic/*.csv"))
demographic_df = pd.concat([pd.read_csv(f) for f in demographic_files], ignore_index=True)
print(f"📋 Demographic: {len(demographic_df):>12,} records")

# Load Biometric Data
biometric_files = sorted(BASE_PATH.glob("api_data_aadhar_biometric/*.csv"))
biometric_df = pd.concat([pd.read_csv(f) for f in biometric_files], ignore_index=True)
print(f"📋 Biometric:   {len(biometric_df):>12,} records")

print("=" * 60)
total = len(enrolment_df) + len(demographic_df) + len(biometric_df)
print(f"📊 TOTAL:       {total:>12,} records")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔧 DATA PREPROCESSING
# ═══════════════════════════════════════════════════════════════════════════════

def preprocess_dataframe(df, dataset_name):
    """Clean and enhance a single dataframe"""
    df = df.copy()
    
    # Parse dates
    df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
    
    # Clean geographic fields
    df['state'] = df['state'].str.strip().str.title()
    df['district'] = df['district'].str.strip().str.title()
    
    # Pincode processing - extract geographic regions
    df['pincode'] = df['pincode'].astype(str).str.zfill(6)
    df['pin_region'] = df['pincode'].str[:3]      # First 3 digits = postal region
    df['pin_subregion'] = df['pincode'].str[:4]   # First 4 digits = sub-region
    
    # Time-based columns
    df['week'] = df['date'].dt.isocalendar().week
    df['month'] = df['date'].dt.month
    df['year_week'] = df['date'].dt.strftime('%Y-W%V')
    df['year_month'] = df['date'].dt.strftime('%Y-%m')
    
    # Create geo_key for easy aggregation
    df['geo_key'] = df['state'] + '|' + df['district']
    
    return df

print("🔧 Preprocessing datasets...")

enrolment_df = preprocess_dataframe(enrolment_df, 'enrolment')
demographic_df = preprocess_dataframe(demographic_df, 'demographic')
biometric_df = preprocess_dataframe(biometric_df, 'biometric')

# Create total activity columns
enrolment_df['total_enrolment'] = (
    enrolment_df['age_0_5'] + 
    enrolment_df['age_5_17'] + 
    enrolment_df['age_18_greater']
)

demographic_df['total_demo_updates'] = (
    demographic_df['demo_age_5_17'] + 
    demographic_df['demo_age_17_']
)

biometric_df['total_bio_updates'] = (
    biometric_df['bio_age_5_17'] + 
    biometric_df['bio_age_17_']
)

print("✅ Preprocessing complete!")

# Display sample
print("\n📋 Sample Enrolment Data:")
enrolment_df.head(3)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 DATA OVERVIEW
# ═══════════════════════════════════════════════════════════════════════════════

# Date range info
print("📅 Date Ranges:")
print(f"   Enrolment:   {enrolment_df['date'].min().strftime('%d-%b-%Y')} → {enrolment_df['date'].max().strftime('%d-%b-%Y')}")
print(f"   Demographic: {demographic_df['date'].min().strftime('%d-%b-%Y')} → {demographic_df['date'].max().strftime('%d-%b-%Y')}")
print(f"   Biometric:   {biometric_df['date'].min().strftime('%d-%b-%Y')} → {biometric_df['date'].max().strftime('%d-%b-%Y')}")

# Geographic coverage
states = set(enrolment_df['state'].unique()) | set(demographic_df['state'].unique()) | set(biometric_df['state'].unique())
districts = set(enrolment_df['district'].unique()) | set(demographic_df['district'].unique()) | set(biometric_df['district'].unique())
pincodes = set(enrolment_df['pincode'].unique()) | set(demographic_df['pincode'].unique()) | set(biometric_df['pincode'].unique())

print(f"\n🗺️  Geographic Coverage:")
print(f"   States:    {len(states)}")
print(f"   Districts: {len(districts)}")
print(f"   Pincodes:  {len(pincodes):,}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🔗 CREATE COMBINED ACTIVITY DATASET
# ═══════════════════════════════════════════════════════════════════════════════

print("📊 Creating combined activity dataset...")

# Aggregate each dataset by geographic and temporal keys
agg_cols = ['date', 'state', 'district', 'geo_key', 'pin_region', 'year_week']

enrol_agg = enrolment_df.groupby(agg_cols)['total_enrolment'].sum().reset_index()
enrol_agg.rename(columns={'total_enrolment': 'enrolment'}, inplace=True)

demo_agg = demographic_df.groupby(agg_cols)['total_demo_updates'].sum().reset_index()
demo_agg.rename(columns={'total_demo_updates': 'demo_updates'}, inplace=True)

bio_agg = biometric_df.groupby(agg_cols)['total_bio_updates'].sum().reset_index()
bio_agg.rename(columns={'total_bio_updates': 'bio_updates'}, inplace=True)

# Merge all datasets
combined_df = enrol_agg.merge(
    demo_agg, on=agg_cols, how='outer'
).merge(
    bio_agg, on=agg_cols, how='outer'
).fillna(0)

# Total activity = sum of all types
combined_df['total_activity'] = (
    combined_df['enrolment'] + 
    combined_df['demo_updates'] + 
    combined_df['bio_updates']
)

print(f"✅ Combined dataset: {len(combined_df):,} records")
print(f"   Unique geo_keys: {combined_df['geo_key'].nunique()}")
print(f"   Time periods: {combined_df['year_week'].nunique()}")

combined_df.head()

---

## 2️⃣ Mobility Signal Index (MSI) Computation

### MSI Formula

$$MSI(A, t) = -\rho(\Delta_{activity,A}, \overline{\Delta_{activity,neighbors}}) \times (1 + S_{spread}) \times \frac{\min(|Z|, 3)}{3}$$

Where:
- $\rho$ = Pearson correlation between location A's change and average neighbor change
- $S_{spread}$ = Proportion of neighbors changing in opposite direction
- $Z$ = Z-score of the activity change (anomaly magnitude)

**Interpretation:**
- **High MSI (> 0.3):** Strong redistribution signal
- **MSI ≈ 0:** No clear pattern
- **Negative MSI:** Correlated movement (everyone moving together)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🗺️  BUILD GEOGRAPHIC NEIGHBOR GRAPH
# ═══════════════════════════════════════════════════════════════════════════════

print("🗺️  Building geographic neighbor relationships...")

# Districts within the same state are considered neighbors
state_districts = combined_df.groupby('state')['district'].unique().to_dict()

district_neighbors = {}
for state, dists in state_districts.items():
    dists = list(dists)
    for d1 in dists:
        key = f"{state}|{d1}"
        district_neighbors[key] = [f"{state}|{d2}" for d2 in dists if d2 != d1]

print(f"✅ Built neighbor graph with {len(district_neighbors)} nodes")
print(f"   Average neighbors per district: {np.mean([len(v) for v in district_neighbors.values()]):.1f}")

# Show example
example_key = list(district_neighbors.keys())[0]
print(f"\n   Example: {example_key}")
print(f"   Neighbors: {district_neighbors[example_key][:5]}...")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📈 COMPUTE TEMPORAL CHANGES
# ═══════════════════════════════════════════════════════════════════════════════

print("📈 Computing temporal activity changes...")

# Aggregate by week and geographic unit
weekly_activity = combined_df.groupby(['year_week', 'geo_key'])['total_activity'].sum().reset_index()

# Pivot to get time series per location
activity_pivot = weekly_activity.pivot(
    index='year_week', 
    columns='geo_key', 
    values='total_activity'
).fillna(0)

# Sort by date
activity_pivot = activity_pivot.sort_index()

print(f"✅ Activity matrix: {activity_pivot.shape[0]} weeks × {activity_pivot.shape[1]} locations")

# Compute percentage changes (week-over-week)
pct_changes = activity_pivot.pct_change().replace([np.inf, -np.inf], np.nan).fillna(0)

# Compute rolling z-scores for anomaly detection
rolling_mean = activity_pivot.rolling(window=4, min_periods=1).mean()
rolling_std = activity_pivot.rolling(window=4, min_periods=1).std().replace(0, 1)
z_scores = (activity_pivot - rolling_mean) / rolling_std

print(f"✅ Computed percentage changes and z-scores")

# Preview
print(f"\n📊 Sample activity trends (first 5 locations, last 5 weeks):")
activity_pivot.iloc[-5:, :5]

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🧮 COMPUTE MOBILITY SIGNAL INDEX (MSI)
# ═══════════════════════════════════════════════════════════════════════════════

print("🧮 Computing Mobility Signal Index...")
print("   This may take a moment for large datasets...")

WINDOW_SIZE = 3  # Number of weeks for rolling correlation

msi_records = []
locations = list(pct_changes.columns)
time_periods = list(pct_changes.index)

processed = 0
for loc in locations:
    # Get neighbors
    neighbors = district_neighbors.get(loc, [])
    valid_neighbors = [n for n in neighbors if n in locations]
    
    if len(valid_neighbors) < 2:
        continue
    
    for t_idx in range(WINDOW_SIZE, len(time_periods)):
        t = time_periods[t_idx]
        t_window = time_periods[t_idx - WINDOW_SIZE:t_idx + 1]
        
        # Get changes for this location in the window
        loc_changes = pct_changes.loc[t_window, loc].values
        
        # Get average neighbor changes in the window
        neighbor_changes = pct_changes.loc[t_window, valid_neighbors].mean(axis=1).values
        
        # MSI Component 1: Inverse correlation
        if np.std(loc_changes) > 0.001 and np.std(neighbor_changes) > 0.001:
            correlation = np.corrcoef(loc_changes, neighbor_changes)[0, 1]
            if np.isnan(correlation):
                correlation = 0
            inverse_corr = -correlation  # Negative correlation = redistribution
        else:
            inverse_corr = 0
        
        # MSI Component 2: Spatial spread factor
        loc_direction = np.sign(loc_changes[-1]) if abs(loc_changes[-1]) > 0.01 else 0
        if loc_direction != 0:
            neighbor_directions = np.sign(pct_changes.loc[t, valid_neighbors].values)
            opposite_count = np.sum(neighbor_directions != loc_direction)
            spatial_spread = opposite_count / len(valid_neighbors)
        else:
            spatial_spread = 0
            opposite_count = 0
        
        # MSI Component 3: Anomaly magnitude (z-score)
        z_magnitude = abs(z_scores.loc[t, loc]) if t in z_scores.index else 0
        z_magnitude = min(z_magnitude, 3)  # Cap at 3
        
        # Final MSI calculation
        msi = inverse_corr * (1 + spatial_spread) * (z_magnitude / 3)
        
        # Store record
        state, district = loc.split('|')
        msi_records.append({
            'time_period': t,
            'state': state,
            'district': district,
            'geo_key': loc,
            'msi_score': msi,
            'inverse_correlation': inverse_corr,
            'spatial_spread': spatial_spread,
            'z_magnitude': z_magnitude,
            'activity_change_pct': loc_changes[-1] * 100,
            'neighbor_change_pct': neighbor_changes[-1] * 100,
            'activity_level': activity_pivot.loc[t, loc],
            'num_neighbors': len(valid_neighbors),
            'neighbors_opposite': opposite_count
        })
    
    processed += 1
    if processed % 200 == 0:
        print(f"   Processed {processed}/{len(locations)} locations...")

# Create MSI DataFrame
msi_df = pd.DataFrame(msi_records)

print(f"\n✅ MSI computation complete!")
print(f"   Total measurements: {len(msi_df):,}")
print(f"   High-MSI events (>0.3): {len(msi_df[msi_df['msi_score'] > 0.3]):,}")

msi_df.head()

---

## 3️⃣ Visualizations 🇮🇳

All visualizations use the **Indian Tricolor theme**:
- 🟠 **Saffron:** High MSI / High activity / Redistribution signals
- ⚪ **White:** Neutral / Baseline
- 🟢 **Green:** Low MSI / Normal activity / Stable patterns

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 VISUALIZATION 1: EXECUTIVE SUMMARY DASHBOARD
# ═══════════════════════════════════════════════════════════════════════════════

print("📊 Creating Executive Summary Dashboard...")

# Key metrics - Focus on POSITIVE MSI (redistribution signals)
high_msi_events = msi_df[msi_df['msi_score'] > 0.3]
high_msi_count = len(high_msi_events)
unique_locations = msi_df['geo_key'].nunique()
max_msi = msi_df['msi_score'].max()

fig_summary = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        "🎯 Redistribution Events (MSI > 0.3)",
        "🗺️ Locations Analyzed",
        "📈 Peak MSI Score",
        "🏆 States with Most Redistribution Events",
        "📊 MSI Score Distribution",
        "🌐 Spatial Spread Distribution"
    ),
    specs=[
        [{"type": "indicator"}, {"type": "indicator"}, {"type": "indicator"}],
        [{"type": "bar"}, {"type": "histogram"}, {"type": "histogram"}]
    ],
    vertical_spacing=0.18,
    horizontal_spacing=0.08
)

# Indicators
fig_summary.add_trace(
    go.Indicator(
        mode="number",
        value=high_msi_count,
        number={'font': {'color': INDIA_COLORS['saffron'], 'size': 48}}
    ),
    row=1, col=1
)

fig_summary.add_trace(
    go.Indicator(
        mode="number",
        value=unique_locations,
        number={'font': {'color': INDIA_COLORS['green'], 'size': 48}}
    ),
    row=1, col=2
)

fig_summary.add_trace(
    go.Indicator(
        mode="number",
        value=max_msi,
        number={'font': {'color': INDIA_COLORS['saffron'], 'size': 48}, 'valueformat': '.4f'}
    ),
    row=1, col=3
)

# Top states BY REDISTRIBUTION EVENT COUNT (not mean MSI)
state_event_counts = high_msi_events.groupby('state').size().sort_values(ascending=False).head(10)
fig_summary.add_trace(
    go.Bar(
        x=state_event_counts.index,
        y=state_event_counts.values,
        marker=dict(
            color=state_event_counts.values,
            colorscale=INDIA_COLORSCALE
        ),
        text=state_event_counts.values,
        textposition='outside',
        textfont=dict(color=INDIA_COLORS['text'], size=10)
    ),
    row=2, col=1
)

# MSI distribution
fig_summary.add_trace(
    go.Histogram(
        x=msi_df['msi_score'],
        nbinsx=50,
        marker=dict(
            color=INDIA_COLORS['saffron'],
            line=dict(color=INDIA_COLORS['saffron_dark'], width=1)
        )
    ),
    row=2, col=2
)

# Spatial spread distribution
fig_summary.add_trace(
    go.Histogram(
        x=msi_df['spatial_spread'],
        nbinsx=30,
        marker=dict(
            color=INDIA_COLORS['green'],
            line=dict(color=INDIA_COLORS['green_dark'], width=1)
        )
    ),
    row=2, col=3
)

fig_summary.update_layout(
    template=INDIA_TEMPLATE,
    title=dict(
        text="🇮🇳 MOBILITY SIGNAL INDEX — EXECUTIVE SUMMARY",
        font=dict(size=28, color=INDIA_COLORS['saffron'])
    ),
    height=650,
    showlegend=False
)

fig_summary.show()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 VISUALIZATION 2: MSI HEATMAP BY STATE & TIME
# ═══════════════════════════════════════════════════════════════════════════════

print("📊 Creating MSI Temporal Heatmap...")

# Aggregate by state and time
heatmap_data = msi_df.groupby(['time_period', 'state'])['msi_score'].mean().reset_index()
heatmap_pivot = heatmap_data.pivot(
    index='state',
    columns='time_period',
    values='msi_score'
).fillna(0)

# Sort states by total MSI
state_order = heatmap_pivot.sum(axis=1).sort_values(ascending=True).index
heatmap_pivot = heatmap_pivot.loc[state_order]

fig_heatmap = go.Figure(data=go.Heatmap(
    z=heatmap_pivot.values,
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    colorscale=INDIA_COLORSCALE,
    colorbar=dict(
        title=dict(text="MSI Score", font=dict(color=INDIA_COLORS['text'])),
        tickfont=dict(color=INDIA_COLORS['text']),
        len=0.9
    ),
    hovertemplate="State: %{y}<br>Week: %{x}<br>MSI: %{z:.4f}<extra></extra>"
))

fig_heatmap.update_layout(
    template=INDIA_TEMPLATE,
    title=dict(
        text="🗓️ MSI Temporal Heatmap: When Did Redistribution Patterns Peak?",
        font=dict(size=22, color=INDIA_COLORS['saffron'])
    ),
    xaxis_title="Week",
    yaxis_title="State",
    height=900,
    xaxis=dict(tickangle=45)
)

fig_heatmap.show()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 VISUALIZATION 3: TOP REDISTRIBUTION HOTSPOTS (RANKED)
# ═══════════════════════════════════════════════════════════════════════════════

print("📊 Creating Hotspot Ranking...")

# Aggregate MSI by location
hotspot_stats = msi_df.groupby(['state', 'district', 'geo_key']).agg({
    'msi_score': ['mean', 'max', 'std', 'count'],
    'inverse_correlation': 'mean',
    'spatial_spread': 'mean',
    'z_magnitude': 'mean',
    'activity_change_pct': 'mean',
    'activity_level': 'mean'
}).reset_index()

hotspot_stats.columns = [
    'state', 'district', 'geo_key',
    'msi_mean', 'msi_max', 'msi_std', 'event_count',
    'avg_inverse_corr', 'avg_spatial_spread', 'avg_z_magnitude',
    'avg_activity_change', 'avg_activity_level'
]

# Composite hotspot score
hotspot_stats['hotspot_score'] = (
    hotspot_stats['msi_mean'] * 0.3 +
    hotspot_stats['msi_max'] * 0.3 +
    hotspot_stats['avg_spatial_spread'] * 0.2 +
    np.log1p(hotspot_stats['event_count']) * 0.1 +
    hotspot_stats['avg_z_magnitude'] / 3 * 0.1
)

hotspot_stats = hotspot_stats.sort_values('hotspot_score', ascending=False)
top_hotspots = hotspot_stats.head(20)

# Create horizontal bar chart
fig_hotspots = go.Figure()

fig_hotspots.add_trace(go.Bar(
    y=[f"{row['district']}, {row['state']}" for _, row in top_hotspots.iterrows()],
    x=top_hotspots['hotspot_score'],
    orientation='h',
    marker=dict(
        color=top_hotspots['hotspot_score'],
        colorscale=INDIA_COLORSCALE,
        line=dict(color=INDIA_COLORS['saffron_dark'], width=1)
    ),
    text=[f"Score: {s:.3f}" for s in top_hotspots['hotspot_score']],
    textposition='inside',
    textfont=dict(color=INDIA_COLORS['background'], size=11, family='Arial Black'),
    hovertemplate=(
        "<b>%{y}</b><br>" +
        "Hotspot Score: %{x:.4f}<br>" +
        "<extra></extra>"
    )
))

fig_hotspots.update_layout(
    template=INDIA_TEMPLATE,
    title=dict(
        text="🎯 TOP 20 REDISTRIBUTION HOTSPOTS (Ranked by MSI)",
        font=dict(size=22, color=INDIA_COLORS['saffron'])
    ),
    xaxis_title="Hotspot Score",
    yaxis_title="",
    height=700,
    showlegend=False,
    yaxis=dict(autorange="reversed")
)

fig_hotspots.show()

# Display table
print("\n🎯 TOP 20 REDISTRIBUTION HOTSPOTS:")
print("=" * 80)
top_hotspots[['district', 'state', 'hotspot_score', 'msi_mean', 'msi_max', 'event_count', 'avg_spatial_spread']].round(4)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 VISUALIZATION 4: TEMPORAL TREND ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════════

print("📊 Creating Temporal Trend Analysis...")

# Aggregate MSI over time
temporal_msi = msi_df.groupby('time_period').agg({
    'msi_score': ['mean', 'std', 'max', 'count'],
    'activity_level': 'sum',
    'geo_key': 'nunique'
}).reset_index()
temporal_msi.columns = ['time_period', 'msi_mean', 'msi_std', 'msi_max', 'event_count', 'total_activity', 'active_locations']

fig_temporal = make_subplots(
    rows=2, cols=1,
    subplot_titles=(
        "📈 MSI Trend Over Time (with confidence band)",
        "📊 Activity Volume & Active Locations"
    ),
    shared_xaxes=True,
    vertical_spacing=0.12
)

# Confidence band (±1 std)
fig_temporal.add_trace(
    go.Scatter(
        x=temporal_msi['time_period'],
        y=temporal_msi['msi_mean'] + temporal_msi['msi_std'],
        mode='lines',
        line=dict(width=0),
        showlegend=False,
        hoverinfo='skip'
    ),
    row=1, col=1
)

fig_temporal.add_trace(
    go.Scatter(
        x=temporal_msi['time_period'],
        y=temporal_msi['msi_mean'] - temporal_msi['msi_std'],
        mode='lines',
        line=dict(width=0),
        fill='tonexty',
        fillcolor='rgba(255, 153, 51, 0.2)',
        showlegend=False,
        hoverinfo='skip'
    ),
    row=1, col=1
)

# Mean MSI line
fig_temporal.add_trace(
    go.Scatter(
        x=temporal_msi['time_period'],
        y=temporal_msi['msi_mean'],
        mode='lines+markers',
        name='Mean MSI',
        line=dict(color=INDIA_COLORS['saffron'], width=3),
        marker=dict(size=8, color=INDIA_COLORS['saffron'])
    ),
    row=1, col=1
)

# Max MSI line
fig_temporal.add_trace(
    go.Scatter(
        x=temporal_msi['time_period'],
        y=temporal_msi['msi_max'],
        mode='lines+markers',
        name='Max MSI',
        line=dict(color=INDIA_COLORS['green'], width=2, dash='dash'),
        marker=dict(size=6, color=INDIA_COLORS['green'])
    ),
    row=1, col=1
)

# Activity bars
fig_temporal.add_trace(
    go.Bar(
        x=temporal_msi['time_period'],
        y=temporal_msi['total_activity'],
        name='Total Activity',
        marker=dict(color=INDIA_COLORS['green'], opacity=0.7)
    ),
    row=2, col=1
)

fig_temporal.update_layout(
    template=INDIA_TEMPLATE,
    title=dict(
        text="📅 TEMPORAL MSI ANALYSIS: When Did Redistribution Patterns Occur?",
        font=dict(size=20, color=INDIA_COLORS['saffron'])
    ),
    height=700,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5
    ),
    xaxis2=dict(tickangle=45)
)

fig_temporal.show()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 VISUALIZATION 5: STATE-WISE MSI COMPARISON
# ═══════════════════════════════════════════════════════════════════════════════

print("📊 Creating State Comparison...")

# State-level statistics
state_stats = msi_df.groupby('state').agg({
    'msi_score': ['mean', 'max', 'std'],
    'activity_level': 'mean',
    'spatial_spread': 'mean',
    'district': 'nunique'
}).reset_index()
state_stats.columns = ['state', 'msi_mean', 'msi_max', 'msi_std', 'avg_activity', 'avg_spread', 'num_districts']
state_stats = state_stats.sort_values('msi_mean', ascending=True)

fig_states = go.Figure()

# Mean MSI bars
fig_states.add_trace(go.Bar(
    y=state_stats['state'],
    x=state_stats['msi_mean'],
    orientation='h',
    name='Mean MSI',
    marker=dict(
        color=state_stats['msi_mean'],
        colorscale=INDIA_COLORSCALE,
        line=dict(width=0)
    ),
    text=[f"{v:.4f}" for v in state_stats['msi_mean']],
    textposition='outside',
    textfont=dict(color=INDIA_COLORS['text'], size=9)
))

# Max MSI markers
fig_states.add_trace(go.Scatter(
    y=state_stats['state'],
    x=state_stats['msi_max'],
    mode='markers',
    name='Max MSI',
    marker=dict(
        size=12,
        color=INDIA_COLORS['chakra_blue'],
        symbol='diamond',
        line=dict(color=INDIA_COLORS['white'], width=1)
    )
))

fig_states.update_layout(
    template=INDIA_TEMPLATE,
    title=dict(
        text="🗺️ STATE-WISE MSI COMPARISON: Which States Show Strongest Signals?",
        font=dict(size=20, color=INDIA_COLORS['saffron'])
    ),
    xaxis_title="MSI Score",
    yaxis_title="",
    height=1000,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        y=1.02,
        xanchor='center',
        x=0.5
    ),
    barmode='overlay'
)

fig_states.show()

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🌊 WAVE PATTERN DETECTION
# ═══════════════════════════════════════════════════════════════════════════════

print("🌊 Detecting wave propagation patterns...")

MIN_DURATION = 3  # Minimum weeks for a wave
MIN_SPREAD = 3    # Minimum districts affected

waves = []
high_msi = msi_df[msi_df['msi_score'] > 0.3].copy()

for state in high_msi['state'].unique():
    state_data = high_msi[high_msi['state'] == state].copy()
    state_data = state_data.sort_values('time_period')
    
    time_periods_list = sorted(state_data['time_period'].unique())
    
    for start_idx, start_time in enumerate(time_periods_list[:-MIN_DURATION]):
        # Get initial districts
        initial = set(state_data[state_data['time_period'] == start_time]['district'].unique())
        
        if len(initial) == 0:
            continue
        
        # Track spread
        affected = {start_time: initial}
        cumulative = initial.copy()
        
        for t in time_periods_list[start_idx + 1:start_idx + MIN_DURATION + 3]:
            if t in state_data['time_period'].values:
                new = set(state_data[state_data['time_period'] == t]['district'].unique())
                affected[t] = new
                cumulative = cumulative | new
        
        # Check if wave pattern
        counts = [len(d) for d in affected.values()]
        
        if len(cumulative) >= MIN_SPREAD and len(counts) > 1 and max(counts) > counts[0]:
            # Found a potential wave
            peak_idx = np.argmax(counts)
            peak_time = list(affected.keys())[peak_idx]
            
            waves.append({
                'state': state,
                'start_time': start_time,
                'peak_time': peak_time,
                'duration': len(affected),
                'origin_districts': list(initial),
                'total_affected': len(cumulative),
                'all_districts': list(cumulative),
                'spread_sequence': {str(k): list(v) for k, v in affected.items()},
                'peak_count': max(counts),
                'wave_score': len(cumulative) * max(counts) / (len(initial) + 1)
            })

waves = sorted(waves, key=lambda x: x['wave_score'], reverse=True)

print(f"\n✅ Detected {len(waves)} wave patterns")

if waves:
    print("\n🌊 TOP 10 WAVE PATTERNS:")
    print("=" * 80)
    for i, wave in enumerate(waves[:10], 1):
        print(f"\n{i}. {wave['state']}")
        print(f"   Timeline: {wave['start_time']} → {wave['peak_time']}")
        print(f"   Origin: {', '.join(wave['origin_districts'][:3])}{'...' if len(wave['origin_districts']) > 3 else ''}")
        print(f"   Affected: {wave['total_affected']} districts | Wave Score: {wave['wave_score']:.2f}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📊 VISUALIZATION 6: WAVE PATTERN ANALYSIS
# ═══════════════════════════════════════════════════════════════════════════════

if waves:
    print("📊 Creating Wave Pattern Visualization...")
    
    # Visualize top wave
    top_wave = waves[0]
    
    fig_wave = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            f"🌊 Wave Score Gauge",
            "📈 District Spread Over Time",
            "🗺️ Affected Districts Timeline",
            "📊 Cumulative Wave Intensity"
        ),
        specs=[
            [{"type": "indicator"}, {"type": "bar"}],
            [{"type": "scatter"}, {"type": "bar"}]
        ]
    )
    
    # Wave score gauge
    fig_wave.add_trace(
        go.Indicator(
            mode="gauge+number",
            value=top_wave['wave_score'],
            title={'text': "Wave Score", 'font': {'color': INDIA_COLORS['text']}},
            gauge={
                'axis': {'range': [0, max(top_wave['wave_score'] * 1.5, 10)]},
                'bar': {'color': INDIA_COLORS['saffron']},
                'bgcolor': INDIA_COLORS['background'],
                'borderwidth': 2,
                'bordercolor': INDIA_COLORS['saffron'],
                'steps': [
                    {'range': [0, top_wave['wave_score']/3], 'color': INDIA_COLORS['green']},
                    {'range': [top_wave['wave_score']/3, top_wave['wave_score']*2/3], 'color': INDIA_COLORS['white']},
                    {'range': [top_wave['wave_score']*2/3, top_wave['wave_score']*1.5], 'color': INDIA_COLORS['saffron']}
                ]
            },
            number={'font': {'color': INDIA_COLORS['saffron']}}
        ),
        row=1, col=1
    )
    
    # District counts over time
    spread_data = top_wave['spread_sequence']
    times = list(spread_data.keys())
    counts = [len(spread_data[t]) for t in times]
    
    fig_wave.add_trace(
        go.Bar(
            x=times,
            y=counts,
            marker=dict(
                color=counts,
                colorscale=INDIA_COLORSCALE
            ),
            text=counts,
            textposition='outside',
            textfont=dict(color=INDIA_COLORS['text'])
        ),
        row=1, col=2
    )
    
    # Timeline scatter
    for i, (t, dists) in enumerate(spread_data.items()):
        fig_wave.add_trace(
            go.Scatter(
                x=[t] * len(dists),
                y=list(range(len(dists))),
                mode='markers+text',
                marker=dict(
                    size=12,
                    color=INDIA_COLORS['saffron'] if i == 0 else INDIA_COLORS['green'],
                    symbol='circle'
                ),
                text=dists,
                textposition='middle right',
                textfont=dict(size=9, color=INDIA_COLORS['text']),
                showlegend=False
            ),
            row=2, col=1
        )
    
    # Cumulative intensity
    cumulative_counts = np.cumsum(counts)
    fig_wave.add_trace(
        go.Bar(
            x=times,
            y=cumulative_counts,
            marker=dict(
                color=cumulative_counts,
                colorscale=[[0, INDIA_COLORS['green']], [1, INDIA_COLORS['saffron']]]
            ),
            text=cumulative_counts,
            textposition='outside',
            textfont=dict(color=INDIA_COLORS['text'])
        ),
        row=2, col=2
    )
    
    fig_wave.update_layout(
        template=INDIA_TEMPLATE,
        title=dict(
            text=f"🌊 WAVE PATTERN ANALYSIS: {top_wave['state']} ({top_wave['start_time']} → {top_wave['peak_time']})",
            font=dict(size=20, color=INDIA_COLORS['saffron'])
        ),
        height=700,
        showlegend=False
    )
    
    fig_wave.show()
else:
    print("⚠️ No significant wave patterns detected with current thresholds")

---

## 4️⃣ Key Findings & Insights

### Interpretation Guidelines

| MSI Score | Interpretation |
|-----------|----------------|
| > 0.5 | **Very Strong** redistribution signal |
| 0.3 - 0.5 | **Strong** redistribution signal |
| 0.1 - 0.3 | **Moderate** signal, may be noise |
| < 0.1 | **Weak/No** significant pattern |

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 📋 FINAL SUMMARY & KEY INSIGHTS
# ═══════════════════════════════════════════════════════════════════════════════

print("=" * 80)
print("🇮🇳 MOBILITY SIGNAL INDEX ANALYSIS — FINAL SUMMARY")
print("=" * 80)

print("\n📊 DATASET OVERVIEW:")
print(f"   • Total records analyzed: {len(enrolment_df) + len(demographic_df) + len(biometric_df):,}")
print(f"   • Time period: {enrolment_df['date'].min().strftime('%b %Y')} - {enrolment_df['date'].max().strftime('%b %Y')}")
print(f"   • States covered: {len(states)}")
print(f"   • Districts analyzed: {msi_df['geo_key'].nunique()}")

print("\n🧮 MSI STATISTICS:")
print(f"   • Total MSI measurements: {len(msi_df):,}")
print(f"   • Average MSI: {msi_df['msi_score'].mean():.4f}")
print(f"   • Maximum MSI: {msi_df['msi_score'].max():.4f}")
print(f"   • High-MSI events (>0.3): {len(msi_df[msi_df['msi_score'] > 0.3]):,}")

print("\n🎯 TOP 5 REDISTRIBUTION HOTSPOTS:")
for i, (_, row) in enumerate(top_hotspots.head(5).iterrows(), 1):
    print(f"   {i}. {row['district']}, {row['state']}")
    print(f"      Score: {row['hotspot_score']:.4f} | Mean MSI: {row['msi_mean']:.4f} | Max MSI: {row['msi_max']:.4f}")

if waves:
    print(f"\n🌊 WAVE PATTERNS DETECTED: {len(waves)}")
    print("   Top wave:")
    print(f"   • State: {waves[0]['state']}")
    print(f"   • Timeline: {waves[0]['start_time']} → {waves[0]['peak_time']}")
    print(f"   • Districts affected: {waves[0]['total_affected']}")
    print(f"   • Wave score: {waves[0]['wave_score']:.2f}")

print("\n" + "=" * 80)
print("⚠️  IMPORTANT DISCLAIMER:")
print("   These findings indicate STATISTICAL PATTERNS consistent with redistribution.")
print("   They do NOT prove actual population movement.")
print("   Possible alternative explanations include:")
print("   • Enrollment camp effects")
print("   • Seasonal variations")
print("   • Administrative changes")
print("   • Data collection variations")
print("=" * 80)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 💾 EXPORT RESULTS
# ═══════════════════════════════════════════════════════════════════════════════

print("💾 Exporting results...")

# Export MSI results
msi_df.to_csv('msi_results.csv', index=False)
print("   ✓ msi_results.csv")

# Export hotspots
hotspot_stats.to_csv('redistribution_hotspots.csv', index=False)
print("   ✓ redistribution_hotspots.csv")

# Export wave patterns
if waves:
    waves_df = pd.DataFrame(waves)
    waves_df.to_csv('wave_patterns.csv', index=False)
    print("   ✓ wave_patterns.csv")

# Save visualizations as HTML
output_dir = Path('msi_visualizations')
output_dir.mkdir(exist_ok=True)

fig_summary.write_html(output_dir / 'summary_dashboard.html')
fig_heatmap.write_html(output_dir / 'msi_heatmap.html')
fig_hotspots.write_html(output_dir / 'hotspot_ranking.html')
fig_temporal.write_html(output_dir / 'temporal_analysis.html')
fig_states.write_html(output_dir / 'state_comparison.html')
if waves:
    fig_wave.write_html(output_dir / 'wave_analysis.html')

print(f"   ✓ All visualizations saved to {output_dir}/")

print("\n✅ Export complete!")

---

## 📚 Technical Appendix

### MSI Algorithm Summary

```
For each location A and time period t:
1. Get activity changes for A over window W
2. Get average activity changes for A's neighbors over window W
3. Compute inverse correlation: ρ_inv = -corr(ΔA, Δneighbors)
4. Compute spatial spread: S = (# neighbors moving opposite) / (# neighbors)
5. Compute anomaly magnitude: Z = (activity - rolling_mean) / rolling_std
6. Final MSI = ρ_inv × (1 + S) × min(|Z|, 3) / 3
```

### Wave Detection Algorithm

```
For each state:
1. Find time periods with high-MSI events
2. For each potential start time:
   a. Track which districts show high MSI
   b. Track spread to new districts in subsequent periods
   c. If total affected ≥ threshold AND pattern shows growth → flag as wave
3. Score waves by: total_affected × peak_count / initial_count
```

### Geographic Neighbor Definition

In absence of explicit coordinates, neighbors are defined as:
- All districts within the same state
- (Optional enhancement: Use pincode prefix similarity for finer granularity)

---

**Created for UIDAI Aadhaar Hackathon 2025**

🇮🇳 *Theme: Indian Tricolor — Saffron • White • Green*